# Support Vector Machines
Support Vector Machines (SVMs) are models for regression and classification tasks. SVM models have two particularly desirable features: robustness in the presence of noisy data and applicability to a variety of data configurations. At its core, a linear SVM model is a hyperplane separating two distinct classes of data (in the case of classification problems), in such a way that the distance between the hyperplane and the nearest training data point (called the margin) is maximized. Vectors that lie on this margin are called support vectors. With the support vectors fixed, perturbations of vectors beyond the margin will not affect the model; this contributes to the model’s robustness. By substituting a kernel function for the usual inner product, one can approximate a large variety of decision boundaries in addition to linear hyperplanes.

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum Database 5.x on GCP (PM demo machine) - direct external IP access
#%sql postgresql://gpadmin@34.67.65.96:5432/madlib

# Greenplum Database 5.x on GCP (PM demo machine) - via tunnel
%sql postgresql://gpadmin@localhost:8000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

u'Connected: gpadmin@madlib'

In [3]:
%sql select madlib.version();

1 rows affected.


version
"MADlib version: 1.17-dev, git revision: rel/v1.16-29-g9fa27e5, cmake configuration time: Mon Oct 7 17:04:14 UTC 2019, build type: release, build system: Linux-3.10.0-957.27.2.el7.x86_64, C compiler: gcc 4.8.5, C++ compiler: g++ 4.8.5"


# Classification
# 1. Create input data set

In [4]:
%%sql 
DROP TABLE IF EXISTS houses;

CREATE TABLE houses (id INT, tax INT, bedroom INT, bath FLOAT, price INT,
            size INT, lot INT);

INSERT INTO houses VALUES   
  (1 ,  590 ,       2 ,    1 ,  50000 ,  770 , 22100),
  (2 , 1050 ,       3 ,    2 ,  85000 , 1410 , 12000),
  (3 ,   20 ,       3 ,    1 ,  22500 , 1060 ,  3500),
  (4 ,  870 ,       2 ,    2 ,  90000 , 1300 , 17500),
  (5 , 1320 ,       3 ,    2 , 133000 , 1500 , 30000),
  (6 , 1350 ,       2 ,    1 ,  90500 ,  820 , 25700),
  (7 , 2790 ,       3 ,  2.5 , 260000 , 2130 , 25000),
  (8 ,  680 ,       2 ,    1 , 142500 , 1170 , 22000),
  (9 , 1840 ,       3 ,    2 , 160000 , 1500 , 19000),
 (10 , 3680 ,       4 ,    2 , 240000 , 2790 , 20000),
 (11 , 1660 ,       3 ,    1 ,  87000 , 1030 , 17500),
 (12 , 1620 ,       3 ,    2 , 118600 , 1250 , 20000),
 (13 , 3100 ,       3 ,    2 , 140000 , 1760 , 38000),
 (14 , 2070 ,       2 ,    3 , 148000 , 1550 , 14000),
 (15 ,  650 ,       3 ,  1.5 ,  65000 , 1450 , 12000);
    
SELECT * FROM houses ORDER BY id;

Done.
Done.
15 rows affected.
15 rows affected.


id,tax,bedroom,bath,price,size,lot
1,590,2,1.0,50000,770,22100
2,1050,3,2.0,85000,1410,12000
3,20,3,1.0,22500,1060,3500
4,870,2,2.0,90000,1300,17500
5,1320,3,2.0,133000,1500,30000
6,1350,2,1.0,90500,820,25700
7,2790,3,2.5,260000,2130,25000
8,680,2,1.0,142500,1170,22000
9,1840,3,2.0,160000,1500,19000
10,3680,4,2.0,240000,2790,20000



# 2. Train linear classification model
Categorical variable is price < $100,0000.

In [5]:
%%sql
DROP TABLE IF EXISTS houses_svm, houses_svm_summary;

SELECT madlib.svm_classification('houses',
                                 'houses_svm',
                                 'price < 100000',
                                 'ARRAY[1, tax, bath, size]'
                           );
SELECT * FROM houses_svm;

Done.
1 rows affected.
1 rows affected.


coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[0.12229100715556, -0.00311209904999331, 0.0729255891679728, 0.00159299038324124]",0.724409814168,4412.03185362,100,15,0,"[False, True]"


# 3. Predict using linear model
We want to predict if house price is less than $100,000. We use the training data set for prediction as well, which is not usual but serves to show the syntax. The predicted results are in the "prediction" column and the actual data is in the "actual" column.

In [6]:
%%sql
DROP TABLE IF EXISTS houses_pred;

SELECT madlib.svm_predict('houses_svm', 
                          'houses', 
                          'id', 
                          'houses_pred');

SELECT *, price < 100000 AS actual FROM houses JOIN houses_pred USING (id) ORDER BY id;

Done.
1 rows affected.
15 rows affected.


id,tax,bedroom,bath,price,size,lot,prediction,decision_function,actual
1,590,2,1.0,50000,770,22100,False,-0.414319248077,True
2,1050,3,2.0,85000,1410,12000,False,-0.753445376631,True
3,20,3,1.0,22500,1060,3500,True,1.82154442156,True
4,870,2,2.0,90000,1300,17500,False,-0.368496489789,True
5,1320,3,2.0,133000,1500,30000,False,-1.45034298564,False
6,1350,2,1.0,90500,820,25700,False,-2.69986500691,True
7,2790,3,2.5,260000,2130,25000,False,-4.9850818531,False
8,680,2,1.0,142500,1170,22000,False,-0.0572120092797,False
9,1840,3,2.0,160000,1500,19000,False,-3.06863449163,False
10,3680,4,2.0,240000,2790,20000,False,-6.73993914924,False


Count the miss-classifications:

In [7]:
%%sql
SELECT COUNT(*) FROM houses_pred JOIN houses USING (id) 
WHERE houses_pred.prediction != (houses.price < 100000);

1 rows affected.


count
5


# 4. Train using Gaussian kernel
Next generate a nonlinear model using a Gaussian kernel. This time we specify the initial step size and maximum number of iterations to run. As part of the kernel parameter, we choose 10 as the dimension of the space where we train SVM. A larger number will lead to a more powerful model but run the risk of overfitting. As a result, the model will be a 10 dimensional vector, instead of 4 as in the case of linear model.

In [8]:
%%sql
DROP TABLE IF EXISTS houses_svm_gaussian, houses_svm_gaussian_summary, houses_svm_gaussian_random;

SELECT madlib.svm_classification( 'houses',
                                  'houses_svm_gaussian',
                                  'price < 100000',
                                  'ARRAY[1, tax, bath, size]',
                                  'gaussian',
                                  'n_components=100',
                                  '',
                                  'init_stepsize=1, max_iter=200'
                           );

SELECT * FROM houses_svm_gaussian;

Done.
1 rows affected.
1 rows affected.


coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[0.174449870282045, 0.221949803360671, -0.402333071003166, -0.262676154168673, 1.13130014096269, -0.477266861255365, 0.831261399413409, 0.603981268320905, 0.470637538053085, 0.504604196295102, 0.0285966246226131, -0.405297458067235, 0.738052957351954, 0.0144918647607145, 0.216536544966497, 0.320016273243399, 0.00232337917551938, -0.0843958273523198, -0.298021033404343, 0.879347233724568, 0.116421698629981, 0.376803883069554, 0.46563131778655, -0.203321710241967, -0.236572471896519, 0.383255178998929, -0.492255525590516, 0.983589649060747, 0.236964019247516, 0.416962283463462, -0.385800872437151, -0.345329823794884, 0.120732524424864, 0.710780837147856, 0.264794396724802, -0.489555512573859, 0.380902730877136, 0.0536009019790365, 0.73200275671809, 0.542826436375729, 0.168712544719772, -0.132541621427206, -0.214051414056401, 0.219073863854421, 0.730947258770631, 0.162460261370712, -0.144429756928593, -0.935475925932472, -0.0297216095002956, 0.193381904334028, 0.448848607315711, -1.18721596180884, 0.482417336135806, -0.52571602337991, -0.818120577090183, 0.260984129515484, -0.557576024742713, -0.83770700093059, -0.253588929795698, -0.463277449925523, -0.0810102775249212, -0.181539340380399, -0.21854707989751, -0.168502090717447, 0.457147867346449, 0.00313147461640386, 0.468228657916444, 0.295498231571813, -0.0862639643663198, -0.252594854475285, 0.732161837953837, 0.621826163021027, 0.821103686828688, -0.0698709773990468, 0.320661634243996, -0.518205856242166, 0.175088291193444, 0.464641905215399, 0.546148979761043, 0.459529900686438, 0.846112959072494, 0.438090206970192, -0.394317794019626, -0.421641334028624, 0.0886399404180068, -0.366570124483957, -0.298487191729675, 0.0355220264779079, -0.009380543657684, 0.510081333996271, -0.368854827597419, 0.216087166764326, -0.411797703281369, -0.163229347789594, 0.229469291437079, -0.164150670617032, 0.0745696677770123, -0.452725636607896, -0.861040652904235, 0.342036947159568]",0.00722931656529,0.0465703228418,177,15,0,"[False, True]"


# 5. Predict using Gaussian model
The predicted results are in the "prediction" column and the actual data is in the "actual" column.

In [9]:
%%sql
DROP TABLE IF EXISTS houses_pred_gaussian;

SELECT madlib.svm_predict('houses_svm_gaussian', 
                          'houses', 
                          'id', 
                          'houses_pred_gaussian');

SELECT *, price < 100000 AS actual FROM houses JOIN houses_pred_gaussian USING (id) ORDER BY id;

Done.
1 rows affected.
15 rows affected.


id,tax,bedroom,bath,price,size,lot,prediction,decision_function,actual
1,590,2,1.0,50000,770,22100,True,1.16840726459,True
2,1050,3,2.0,85000,1410,12000,True,1.12629659629,True
3,20,3,1.0,22500,1060,3500,True,1.03236268569,True
4,870,2,2.0,90000,1300,17500,True,1.17721999735,True
5,1320,3,2.0,133000,1500,30000,False,-1.38210027041,False
6,1350,2,1.0,90500,820,25700,True,1.16225381842,True
7,2790,3,2.5,260000,2130,25000,False,-1.28234134504,False
8,680,2,1.0,142500,1170,22000,False,-1.14421198085,False
9,1840,3,2.0,160000,1500,19000,False,-1.04766021862,False
10,3680,4,2.0,240000,2790,20000,False,-1.0549708301,False


Count the miss classifications.  Note this produces a more accurate result than the linear case for this small data set:

In [10]:
%%sql
SELECT COUNT(*) FROM houses_pred_gaussian JOIN houses USING (id) 
WHERE houses_pred_gaussian.prediction != (houses.price < 100000);

1 rows affected.


count
0


# 6.   Balancing data sets
In the case of an unbalanced class-size dataset, use the 'balanced' parameter to classify when building the model:

In [11]:
%%sql
DROP TABLE IF EXISTS houses_svm_gaussian, houses_svm_gaussian_summary, houses_svm_gaussian_random;

SELECT madlib.svm_classification( 'houses',
                                  'houses_svm_gaussian',
                                  'price < 150000',
                                  'ARRAY[1, tax, bath, size]',
                                  'gaussian',
                                  'n_components=100',
                                  '',
                                  'init_stepsize=1, max_iter=200, class_weight=balanced'
                           );

SELECT * FROM houses_svm_gaussian;

Done.
1 rows affected.
1 rows affected.


coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[0.0304283785262617, -0.31252256022358, -0.786594823395088, -0.213462165447043, -0.205921127201806, -0.160495098525847, -0.0572558143143236, 0.183220396715039, 0.464792217394048, 0.481483244258389, -0.517643614876286, -0.978683507901074, 0.16817691490251, -0.646816511296802, 0.186375390577986, 0.680958110801916, -1.08232636410793, -0.945175946274317, -0.491574430145303, -0.0814842224959727, -0.0190008166655649, 0.0481772284935738, -0.163702000177582, -0.674004897487282, 0.113518490341767, -0.638187403343937, 0.526552063250668, -0.274100343388661, 0.354431317955514, -0.428444014517539, 0.0946683130713131, -0.239646558966188, -0.288975110225114, 0.277634287723891, 0.109083762491799, -0.590472152297871, 0.30239084357163, -0.644378259476824, 0.518616701508965, -0.0310448850251757, -0.0616074328876328, -0.815709238025655, 0.533952545147382, -0.27885652806791, -0.169816368423772, 0.501969605761016, -0.0453904283532783, -0.296542126733526, -0.6744641877468, -1.04295422716397, 0.0998805368884473, -0.0581387461992054, 0.226951693444486, 0.183293767588985, -0.506636260378821, -0.182587120340722, -0.632707861890101, 0.165980897095258, -0.918139789600548, -0.770637770944717, 0.945310986017951, -1.02669717302724, 0.608102258578773, 0.22613081465608, 0.141010992575908, -0.154732938082785, 0.673276713057085, -0.252979879432229, 0.373696371450733, -0.204550990421942, 0.934207962184333, -0.306025798509274, 0.812798367835215, -0.455532753255022, 0.125622634105054, 0.367604443133276, 0.55994486569804, 0.0753886324541624, 0.77813576524246, -0.166034479090781, 0.94565093122238, -0.296006132277826, -0.265234940757184, 0.416497397310383, 0.731437706958231, 0.270984781299447, -0.428663581410429, -0.312686040931128, -0.773226583138831, 0.598583108690796, -1.09057460676235, -0.685811206980829, 0.320672030715493, -0.216001011689699, -0.467434588986512, -0.412477454386901, 0.437774085441529, 0.177397734130052, -0.621856758100873, 0.965785439434402]",0.00892975733073,0.0517583539075,177,15,0,"[False, True]"


# Regression
# 1. Create input data set
For regression we use part of the well known abalone data set https://archive.ics.uci.edu/ml/datasets/abalone :

In [12]:
%%sql
DROP TABLE IF EXISTS abalone;

CREATE TABLE abalone (id INT, sex TEXT, length FLOAT, diameter FLOAT, height FLOAT, rings INT);

INSERT INTO abalone VALUES
(1,'M',0.455,0.365,0.095,15),
(2,'M',0.35,0.265,0.09,7),
(3,'F',0.53,0.42,0.135,9),
(4,'M',0.44,0.365,0.125,10),
(5,'I',0.33,0.255,0.08,7),
(6,'I',0.425,0.3,0.095,8),
(7,'F',0.53,0.415,0.15,20),
(8,'F',0.545,0.425,0.125,16),
(9,'M',0.475,0.37,0.125,9),
(10,'F',0.55,0.44,0.15,19),
(11,'F',0.525,0.38,0.14,14),
(12,'M',0.43,0.35,0.11,10),
(13,'M',0.49,0.38,0.135,11),
(14,'F',0.535,0.405,0.145,10),
(15,'F',0.47,0.355,0.1,10),
(16,'M',0.5,0.4,0.13,12),
(17,'I',0.355,0.28,0.085,7),
(18,'F',0.44,0.34,0.1,10),
(19,'M',0.365,0.295,0.08,7),
(20,'M',0.45,0.32,0.1,9);

SELECT * FROM abalone ORDER BY id;

Done.
Done.
20 rows affected.
20 rows affected.


id,sex,length,diameter,height,rings
1,M,0.455,0.365,0.095,15
2,M,0.35,0.265,0.09,7
3,F,0.53,0.42,0.135,9
4,M,0.44,0.365,0.125,10
5,I,0.33,0.255,0.08,7
6,I,0.425,0.3,0.095,8
7,F,0.53,0.415,0.15,20
8,F,0.545,0.425,0.125,16
9,M,0.475,0.37,0.125,9
10,F,0.55,0.44,0.15,19


# 2. Train linear regression model

In [13]:
%%sql
DROP TABLE IF EXISTS abalone_svm_regression, abalone_svm_regression_summary;

SELECT madlib.svm_regression('abalone',
                             'abalone_svm_regression',
                             'rings',
                             'ARRAY[1, length, diameter, height]'
                           );

SELECT * FROM abalone_svm_regression;

Done.
1 rows affected.
1 rows affected.


coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[1.998949892503, 0.918517335088235, 0.712125758488304, 0.229379426728093]",8.29033306424,23.2251777867,100,20,0,[None]


# 3. Predict using linear model

In [14]:
%%sql
DROP TABLE IF EXISTS abalone_regr;

SELECT madlib.svm_predict('abalone_svm_regression',
                          'abalone', 
                          'id', 
                          'abalone_regr');

SELECT * FROM abalone JOIN abalone_regr USING (id) ORDER BY id;

Done.
1 rows affected.
20 rows affected.


id,sex,length,diameter,height,rings,prediction,decision_function
1,M,0.455,0.365,0.095,15,2.69859222736,2.69859222736
2,M,0.35,0.265,0.09,7,2.52978843419,2.52978843419
3,F,0.53,0.42,0.135,9,2.81582312127,2.81582312127
4,M,0.44,0.365,0.125,10,2.69169585013,2.69169585013
5,I,0.33,0.255,0.08,7,2.50200303563,2.50200303563
6,I,0.425,0.3,0.095,8,2.624748533,2.624748533
7,F,0.53,0.415,0.15,20,2.81570318388,2.81570318388
8,F,0.545,0.425,0.125,16,2.83086771582,2.83086771582
9,M,0.475,0.37,0.125,9,2.72740458565,2.72740458565
10,F,0.55,0.44,0.15,19,2.85187667455,2.85187667455


RMS error:

In [15]:
%%sql
SELECT SQRT(AVG((rings-prediction)*(rings-prediction))) as rms_error FROM abalone 
JOIN abalone_regr USING (id);

1 rows affected.


rms_error
9.08842735715


# 4. Train using Gaussian model

In [16]:
%%sql
DROP TABLE IF EXISTS abalone_svm_gaussian_regression, abalone_svm_gaussian_regression_summary, abalone_svm_gaussian_regression_random;

SELECT madlib.svm_regression( 'abalone',
                              'abalone_svm_gaussian_regression',
                              'rings',
                              'ARRAY[1, length, diameter, height]',
                              'gaussian',
                              'n_components=100',
                              '',
                              'init_stepsize=1, max_iter=200'
                           );

SELECT * FROM abalone_svm_gaussian_regression;

Done.
1 rows affected.
1 rows affected.


coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[1.43663248508461, 0.69548130777053, -0.667820070034776, 0.353629960777887, 0.90572382703018, -1.35192985512522, 1.33643594466947, -1.71808935250261, 0.244907459899655, -1.16083721399717, 0.529533556029462, -1.47565163746093, 0.980976366290674, -1.57625949227502, 0.725799354053906, 0.621521219305506, -0.460388482856766, 1.62409092636038, -1.57065529672944, -1.04776403679369, -1.57215274912702, 0.414786913802637, 0.338630545155005, 1.67994043684046, 0.510271334529243, 1.56926726571838, -0.266055554149799, -1.58512365036022, -0.537619986488822, -1.53123126187998, 0.432059794750374, 1.10164971751016, -1.63470052461906, 1.37210730848619, 0.69195070273926, -1.51550206095593, -0.879588483736812, -0.37806986511409, -1.02110166282495, -0.905287480716121, -0.0893285495393008, -0.576435902131671, 1.24002228080107, -0.817209795372, 0.522281410113176, 0.156847005593254, -0.872148823590253, -1.43633009947157, -0.804330453999928, -0.926921599832849, -0.211959168386152, -0.804358663674064, -0.616861771161596, 1.74018724759497, -1.16264563996159, -1.63930487981442, -1.82413119548892, 0.794011977984717, 1.1698395008002, 1.61482847146405, 0.205981267231647, -0.458375714158526, -0.867267996489895, -0.308286903981698, 1.58324830425497, -0.827831442995643, -1.47580575745441, -0.445332216554912, 1.16538304159597, 1.37221102799479, 1.57289281402231, -0.66649715533837, -0.476569227105178, 0.29665562590112, 1.15590670076679, 1.46291613083677, 0.79992035125813, 1.09721250793166, -1.39887016649551, 0.190329304832203, 1.28663304442308, 1.42061537940844, -1.03946229472351, -0.556121905835198, 0.341622218455033, -0.899369783669295, 1.61878187535374, 1.57583095826418, -0.71553739367934, -1.59696136184546, -1.56192177099981, -0.174158730591109, 1.19626739396989, -1.37221525222703, 1.59932192137657, 0.435584312299149, 1.38585223624462, 0.449219561014516, -1.61927162148616, -1.76886002780722]",2.60162532006,1.04369725716,166,20,0,[None]


# 5. Predict using Gaussian model

In [17]:
%%sql
DROP TABLE IF EXISTS abalone_gaussian_regr;

SELECT madlib.svm_predict('abalone_svm_gaussian_regression', 
                          'abalone', 
                          'id', 
                          'abalone_gaussian_regr');

SELECT * FROM abalone JOIN abalone_gaussian_regr USING (id) ORDER BY id;

Done.
1 rows affected.
20 rows affected.


id,sex,length,diameter,height,rings,prediction,decision_function
1,M,0.455,0.365,0.095,15,9.98603457721,9.98603457721
2,M,0.35,0.265,0.09,7,9.57759650003,9.57759650003
3,F,0.53,0.42,0.135,9,10.2223107905,10.2223107905
4,M,0.44,0.365,0.125,10,9.97689991817,9.97689991817
5,I,0.33,0.255,0.08,7,9.49509568417,9.49509568417
6,I,0.425,0.3,0.095,8,9.82953290649,9.82953290649
7,F,0.53,0.415,0.15,20,10.2265451261,10.2265451261
8,F,0.545,0.425,0.125,16,10.2457829829,10.2457829829
9,M,0.475,0.37,0.125,9,10.0580169046,10.0580169046
10,F,0.55,0.44,0.15,19,10.2825919797,10.2825919797


Compute the RMS error.  Note this produces a more accurate result than the linear case for this small data set:

In [18]:
%%sql
SELECT SQRT(AVG((rings-prediction)*(rings-prediction))) as rms_error FROM abalone 
JOIN abalone_gaussian_regr USING (id);

1 rows affected.


rms_error
3.75666107851


# 6. Cross validation
Let's run cross validation for different initial step sizes and lambda values:

In [19]:
%%sql
DROP TABLE IF EXISTS abalone_svm_gaussian_regression, abalone_svm_gaussian_regression_summary, 
abalone_svm_gaussian_regression_random, abalone_svm_gaussian_regression_cv;

SELECT madlib.svm_regression( 'abalone',
                              'abalone_svm_gaussian_regression',
                              'rings',
                              'ARRAY[1, length, diameter, height]',
                              'gaussian',
                              'n_components=100',
                              '',
                              'init_stepsize=[0.01,1], n_folds=3, max_iter=200, lambda=[0.01, 0.1, 0.5], validation_result=abalone_svm_gaussian_regression_cv'
                           );

SELECT * FROM abalone_svm_gaussian_regression;

Done.
1 rows affected.
1 rows affected.


coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[1.43663141048166, 0.69590902120895, -0.667071782085283, 0.353774339959869, 0.905828289634807, -1.35169013920124, 1.33646358226853, -1.7180201996699, 0.24481068329437, -1.16069594203275, 0.529737373827505, -1.47546535187187, 0.980691636646386, -1.57624210352963, 0.725851837240663, 0.621725612776311, -0.460138729733476, 1.62396471237433, -1.57063525564175, -1.04781069512703, -1.57213080053901, 0.415306056428304, 0.338679918908941, 1.6801320999485, 0.510208152821366, 1.56936887737261, -0.266001406651365, -1.58507523905296, -0.537812259067129, -1.5312628382147, 0.432237469443731, 1.10170029776598, -1.63469708889792, 1.3721951919766, 0.691837885175724, -1.5155853305037, -0.879575274457664, -0.378046928285422, -1.02084583660482, -0.905052544480774, -0.0895711922931681, -0.576177025418454, 1.23999477285708, -0.817661938486405, 0.522699926739854, 0.157003593221511, -0.871879320669594, -1.43641645044609, -0.804548716520958, -0.926662211340699, -0.211709523145134, -0.804204137088408, -0.616360942183748, 1.74003514481556, -1.16277414575498, -1.63929118823194, -1.82407340498873, 0.794042280007498, 1.17011142340466, 1.61469087973011, 0.206231124021691, -0.458287050727443, -0.86716709482106, -0.308198290202033, 1.58323228771844, -0.827377112621467, -1.47575759821081, -0.445203431415296, 1.16551153170496, 1.37211944359085, 1.57289635294413, -0.666315273550231, -0.476600127294968, 0.296662747500788, 1.15584301004967, 1.46263374958392, 0.799907772688504, 1.09708642804713, -1.39887637044597, 0.190356100346786, 1.28636855084754, 1.42039838933069, -1.03949687018857, -0.555861325085124, 0.341382237642365, -0.899593443791824, 1.61885754858329, 1.57586783372196, -0.71510056870506, -1.59701588256911, -1.56188454682269, -0.174291105678036, 1.19615194057151, -1.37224827664802, 1.59933543202374, 0.435995073036911, 1.38582746743955, 0.449261405728548, -1.61911544479704, -1.7689529702645]",2.60165100186,1.04370217846,165,20,0,[None]


View the summary table showing the final model parameters are those that produced 
the lowest error in the cross validation runs:

In [20]:
%sql SELECT * FROM abalone_svm_gaussian_regression_summary;

1 rows affected.


method,version_number,source_table,model_table,dependent_varname,independent_varname,kernel_func,kernel_params,grouping_col,optim_params,reg_params,num_all_groups,num_failed_groups,total_rows_processed,total_rows_skipped
SVR,1.17-dev,abalone,abalone_svm_gaussian_regression,rings,"ARRAY[1, length, diameter, height]",gaussian,"gamma=0.25, n_components=100,random_state=1, fit_intercept=False, fit_in_memory=True",NULL,"init_stepsize=1.0, decay_factor=0.9, max_iter=200, tolerance=1e-10, epsilon=0.01, eps_table=, class_weight=","lambda=0.01, norm=l2, n_folds=3",1,0,20,0


View the values for cross validation:

In [21]:
%%sql
SELECT * FROM abalone_svm_gaussian_regression_cv;

6 rows affected.


init_stepsize,lambda,mean_score,std_dev_score
0.01,0.01,-10.6826786731,2.89736952916
0.01,0.5,-10.6896583532,2.89700539715
1.0,0.1,-3.81154741847,2.20160973525
0.01,0.1,-10.6839690886,2.89730222282
1.0,0.01,-3.62237265846,2.30389631902
1.0,0.5,-3.9659217506,2.24231164049


# 7. Predict using cross-validated Gaussian regression model:

In [22]:
%%sql
DROP TABLE IF EXISTS abalone_gaussian_regr;
SELECT madlib.svm_predict('abalone_svm_gaussian_regression', 
                          'abalone', 
                          'id', 
                          'abalone_gaussian_regr');

Done.
1 rows affected.


svm_predict
""


Compute the RMS error. Note this produces a more accurate result than the previous run with the Gaussian kernel:

In [23]:
%%sql
SELECT SQRT(AVG((rings-prediction)*(rings-prediction))) as rms_error FROM abalone 
JOIN abalone_gaussian_regr USING (id);

1 rows affected.


rms_error
3.7567001982


# Novelty detection 
# 1. Train a non-linear one-class SVM
Use a Gaussian kernel using the housing data set. Note that the dependent variable is not a parameter for one-class:

In [24]:
%%sql
DROP TABLE IF EXISTS houses_one_class_gaussian, houses_one_class_gaussian_summary, houses_one_class_gaussian_random;

SELECT madlib.svm_one_class('houses',
                            'houses_one_class_gaussian',
                            'ARRAY[1,tax,bedroom,bath,size,lot,price]',
                            'gaussian',
                            'gamma=0.5,n_components=55, random_state=3',
                            NULL,
                            'max_iter=100, init_stepsize=10,lambda=10, tolerance=0'
                            );

SELECT * FROM houses_one_class_gaussian;

Done.
1 rows affected.
1 rows affected.


coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[0.0207901288823711, -0.00103437489314969, 0.00407820868429805, 0.0274910360546609, 0.0105696547048294, -0.00313332466259033, -0.0216703145014011, 0.0363248037825208, -0.0211400498166549, -0.00827402232219555, 0.0265909439934851, 0.0282462482323058, -0.0407407195393746, 0.0191290942177852, -0.00313542082923064, -0.0191740603622109, 0.0143626646548982, -0.0620527674181034, -0.000319831622794402, 0.00388104709972051, 0.00248129433065678, 0.00764915273571186, 0.014492283562898, 0.0184730815984353, -0.00745840880633255, -0.0232208663374367, -0.010724056217189, 0.00541494627043399, 0.0150679846777238, 0.0204022414812525, -0.0294626167089617, -0.00399506510201406, -0.0231139983460727, 0.0242203153309423, -0.0421196963278802, 0.0112202149916885, -0.00720876723524249, 0.0213674589734111, -0.00260107056222295, -0.0130652059444514, 0.0710580616012718, 0.0519822855717347, 0.00961050532247376, 0.0390561950837254, -0.0152620688050253, 0.0100336750737295, 0.0632488712630204, -0.0549714494076944, -0.007684860916257, 0.0322104572263339, -0.00832311210931705, 0.0279669244721609, 0.0455147539995411, -0.0639670005155479, -0.00965055072583972, 0.00648588125681694]",0.944029805394,14.5264894419,100,16,-1,"[-1.0, 1.0]"


# 2. Create test data
For the novelty detection using one-class, let's create a test data set using the last 3 values from the training set plus an outlier at the end (10x price):

In [25]:
%%sql
DROP TABLE IF EXISTS houses_one_class_test;

CREATE TABLE houses_one_class_test (id INT, tax INT, bedroom INT, bath FLOAT, price INT,
            size INT, lot INT);

INSERT INTO houses_one_class_test VALUES   
 (1 , 3100 ,       3 ,    2 , 140000 , 1760 , 38000),
 (2 , 2070 ,       2 ,    3 , 148000 , 1550 , 14000),
 (3 ,  650 ,       3 ,  1.5 ,  65000 , 1450 , 12000),
 (4 ,  650 ,       3 ,  1.5 ,  650000 , 1450 , 12000);
      
SELECT * FROM houses_one_class_test ORDER BY id;

Done.
Done.
4 rows affected.
4 rows affected.


id,tax,bedroom,bath,price,size,lot
1,3100,3,2.0,140000,1760,38000
2,2070,2,3.0,148000,1550,14000
3,650,3,1.5,65000,1450,12000
4,650,3,1.5,650000,1450,12000


# 3. Predict using Gaussian one-class novelty detection model
Result shows the last row predicted to be novel:

In [26]:
%%sql 
DROP TABLE IF EXISTS houses_pred;

SELECT madlib.svm_predict('houses_one_class_gaussian', 
                          'houses_one_class_test', 
                          'id', 
                          'houses_pred');

SELECT * FROM houses_one_class_test JOIN houses_pred USING (id) ORDER BY id;

Done.
1 rows affected.
4 rows affected.


id,tax,bedroom,bath,price,size,lot,prediction,decision_function
1,3100,3,2.0,140000,1760,38000,1.0,0.0662278474212
2,2070,2,3.0,148000,1550,14000,1.0,0.092124936453
3,650,3,1.5,65000,1450,12000,1.0,0.03415206006
4,650,3,1.5,650000,1450,12000,-1.0,-0.0131918729845
